In [ ]:
!git clone https://github.com/iglu-contest/iglu
!apt-get -qq install openjdk-8-jdk xvfb > /dev/null
!update-alternatives --config java
!sudo add-apt-repository ppa:openjdk-r/ppa
!sudo apt-get update
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
!pip uninstall -y iglu && pip install git+https://github.com/iglu-contest/iglu.git
!pip install gym==0.18.3

In [ ]:
# exec this cell ONLY in colab
!wget -q https://raw.githubusercontent.com/iglu-contest/tutorials/main/env/colab_setup.sh -O - | sh > /dev/null 2>&1
!pip install -q pyvirtualdisplay
from pyvirtualdisplay import Display
disp = Display(backend="xvnc", size=(800, 600))
disp.start();
# for local notebooks instead launch jupyter as: xvfb-run -s "-screen 0 640x480x24" jupyter ...

In [ ]:
import iglu
import gym
from iglu.tasks import RandomTasks
from iglu.tasks.task_set import TaskSet

env = gym.make('IGLUSilentBuilder-v0')
obs = env.reset()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
downloading task dataset into /root/.iglu/data: 258976it [00:02, 104615.09it/s]


In [ ]:
targets = []
chats = []
name_targets = []
chat_embeddings = []
for i in range(1,156):
  if ('C'+str(i)) == 'C38': continue
  name_targets.append('C'+str(i))
  targets.append(TaskSet(preset=['C'+str(i)]).sample().target_grid)
  chats.append(TaskSet(preset=['C'+str(i)]).sample().chat)

## Augmentation chats throw colorizing

In [ ]:
colours = ["blue","yellow","green","orange","purple","red"]

In [ ]:
import re

augmented_targets = [] 
augmented_chats = []
augmented_target_name = []
chat_set = set()
target_set = set()

for c1 in range(len(colours)):
  pattern = colours[c1]
  for c2 in range(len(colours)):
    if (colours[c1] == colours[c2]): continue
    for i in range(len(targets)):
      new_chat = re.sub(pattern,colours[c2],chats[i])
      new_target = targets[i].copy()
      for j in range(9):
        for k in range(11):
          for m in range(11):
            if(new_target[j][k][m] == c1 + 1):
              new_target[j][k][m] = c2 + 1
      if(new_chat not in chat_set):
        chat_set.add(new_chat)
        augmented_chats.append(new_chat)
        augmented_targets.append(new_target)
        augmented_target_name.append(name_targets[i])

In [ ]:
len(augmented_chats),len(augmented_target_name),len(augmented_targets)

(2845, 2845, 2845)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Random delete messages of builder from chat

In [ ]:
def del_random_builder_messages(chat):
  length = 0
  lines=[]
  lines = chat.split('\n')
  new_chat = lines.copy()
  for i in range(len(new_chat)):
    if new_chat[i][:9]=="<Builder>":
      length += 1
  if(length > 0):
    lines_to_del = np.random.randint(0,length)
    if(length > 1 and lines_to_del == 0):
      lines_to_del = 1
    for i in range(lines_to_del):
      builder_arr = []
      for i in range(len(new_chat)):
        if new_chat[i][:9]=="<Builder>":
          builder_arr.append(i)
      index_to_del = np.random.randint(0,len(builder_arr))
      new_chat.pop(builder_arr[index_to_del])
    return new_chat
  else:
    return new_chat


In [ ]:
random_augmented_targets = [] 
random_augmented_chats = []
random_augmented_target_name = []
random_chat_set = set()
random_target_set = set()
for j in range(10):
  for i in range(len(augmented_chats)):
    lines = del_random_builder_messages(augmented_chats[i])
    new_chat = ""
    for line in lines:
      new_chat += line + "\n"
    new_chat = new_chat[:len(new_chat)-1]
    if(new_chat not in chat_set):
      chat_set.add(new_chat)
      random_augmented_chats.append(new_chat)
      random_augmented_targets.append(augmented_targets[i])
      random_augmented_target_name.append(augmented_target_name[i])
len(random_augmented_chats)

202464

## Delete all builder messages

In [ ]:
def del_all_builder_messages(chat):
  length = 0
  lines=[]
  lines = chat.split('\n')
  new_chat = lines.copy()
  for i in range(len(new_chat)):
    if new_chat[i][:9]=="<Builder>":
      length += 1
  if(length > 0):
    for i in range(length):
      builder_arr = []
      for i in range(len(new_chat)):
        if new_chat[i][:9]=="<Builder>":
          builder_arr.append(i)
      index_to_del = 0
      new_chat.pop(builder_arr[index_to_del])
    return new_chat
  else:
    return new_chat

In [ ]:
without_builder_augmented_targets = [] 
without_builder_augmented_chats = []
without_builder_augmented_target_name = []
without_builder_chat_set = set()
without_builder_target_set = set()
for j in range(1):
  for i in range(len(augmented_chats)):
    lines = del_all_builder_messages(augmented_chats[i])
    new_chat = ""
    for line in lines:
      new_chat += line + "\n"
    new_chat = new_chat[:len(new_chat)-1]
    if(new_chat not in without_builder_chat_set):
      without_builder_chat_set.add(new_chat)
      without_builder_augmented_chats.append(new_chat)
      without_builder_augmented_targets.append(augmented_targets[i])
      without_builder_augmented_target_name.append(augmented_target_name[i])

len(without_builder_augmented_chats)

2825

## Fragmented Chats

In [ ]:
fragmented_augmented_targets = [] 
fragmented_augmented_chats = []
fragmented_augmented_target_name = []
fragmented_chat_set = set()

for c1 in range(len(colours)):
  pattern = colours[c1]
  for c2 in range(len(colours)):
    if (colours[c1] == colours[c2]): continue
    for i in range(len(targets)):
      new_chat = re.sub(pattern,colours[c2],chats[i])
      first_part = new_chat[:int(len(new_chat)/2)]
      second_part = new_chat[int(len(new_chat)/2):]
      new_target = targets[i].copy()
      for j in range(9):
        for k in range(11):
          for m in range(11):
            if(new_target[j][k][m] == c1 + 1):
              new_target[j][k][m] = c2 + 1
      if(first_part not in fragmented_chat_set and second_part not in fragmented_chat_set):
        fragmented_chat_set.add(first_part)
        fragmented_chat_set.add(second_part)
        fragmented_augmented_chats.append(first_part)
        fragmented_augmented_targets.append(new_target)
        fragmented_augmented_target_name.append(name_targets[i])
        fragmented_augmented_chats.append(second_part)
        fragmented_augmented_targets.append(new_target)
        fragmented_augmented_target_name.append(name_targets[i])
len(fragmented_augmented_chats),len(fragmented_augmented_target_name),len(fragmented_augmented_targets)

(3902, 3902, 3902)